In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df1 = pd.read_csv('/kaggle/input/zb-da9-competition/Special_Sale_Calendar.csv') #마케팅 프로모션 기간 정보 (2015/2016 일-월 기간정보)
df2 = pd.read_csv('/kaggle/input/zb-da9-competition/ConsumerElectronics.csv') #거래이력 데이터_메인
df3 = pd.read_csv('/kaggle/input/zb-da9-competition/NPS_Stockindex.csv')  # NPS(고객충성도 지표), Stock Index(주가 지수) 정보 (15.07~16.06 /1년간)
df4 = pd.read_csv('/kaggle/input/zb-da9-competition/Media Investment.csv') #광고 및 투자 내역 (년/월/총투자액/매체별 투자액)

df2.head()

# 데이터 전처리

In [ ]:
df2.info()

In [ ]:
df2['product_procurement_sla'] = df2['product_procurement_sla'].replace(-1, 0)
df2['gmv'] = pd.to_numeric(df2['gmv'], errors='coerce').fillna(0).astype(float)

In [ ]:
columns_to_select = [
    'fsn_id', 'order_date', 'order_id', 'order_item_id', 'gmv', 'units', 
    's1_fact.order_payment_type', 'sla', 'cust_id', 
    'product_analytic_category', 'product_analytic_sub_category', 
    'product_analytic_vertical', 'product_mrp', 'product_procurement_sla'
]


df= df2[columns_to_select]
df.head()

In [ ]:
df.info()

* 이상치 제거 후 분석에 필요한 컬럼(fsn_id / order_date / order_id / order_item_id / gmv / units / s1_fact.order_payment_type / sla / cust_id / product_analytic_category / product_analytic_sub_category / product_analytic_vertical / product_mrp / product_procurement_sla)으로 새로운 데이터프레임 생성 (df)
_____________
* 1,648,824행* 14열

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df['order_date'] = pd.to_datetime(df['order_date'])

df['InvoiceDateFirst'] = df.groupby(['cust_id'])['order_date'].transform('min')
df['order_date'] = pd.to_datetime(df['order_date'])
df['InvoiceDateFirst'] = pd.to_datetime(df['InvoiceDateFirst'])

year_diff = df['order_date'].dt.year - df['InvoiceDateFirst'].dt.year
month_diff = df['order_date'].dt.month - df['InvoiceDateFirst'].dt.month

month_diff = month_diff + (year_diff * 12)
df['cohortindex'] = month_diff + 1

df['cohortindex'].value_counts()


df['InvoiceDateFirstYM'] = df['InvoiceDateFirst'].astype(str).str[:7]
# 잔존수
cohort_count = df.groupby(['InvoiceDateFirstYM', 'cohortindex'])['cust_id'].nunique().unstack()

plt.figure(figsize=(12, 8))
sns.heatmap(cohort_count, cmap = 'Greens', annot=True, fmt = ".0f")


* 15년 8월 이후로 고객 충성도 지표가 크게 하락하였다. 약간의 회복 추세를 보이고 있으나 여전히 증감을 반복하며 불안정한 지표를 보이고 있다. 이커머스 사업의 안정적인 성장을 위해 고객 충성도 향상을 위한 데이터 분석을 진행한다.
____________
**코호트 분석**
* 15년 5월~8월은 데이터 이상 또는 사업 초기로 분석하기에 적절하지 않음
* 16년 7월 25일까지 데이터가 존재하는데 7월 데이터는 불완전하므로 제외하고, 재구매율 확인을 위해서는 최초 주문한 달부터 3개월이 경과한 후의 데이터가 있는 15년 9월~16년 3월동안의 구매 데이터를 대상으로 분석
* 15년 9월~11월 : 신규 유입고객 대비 3개월 후 재구매 고객이 급감하였음 (400명대/10명대)
* 15년 12월~ 16년 3월 : 신규 유입 고객 대비 3개월 후 재구매 고객 수가 비교적 유지되고 있음 (2~3,000명대)
> 위 두 경우를 그룹으로 나누어 해당 기간 유입 고객간의 차이를 알아본다.

# 데이터 EDA - 고객 그룹(1, 2)별 비교

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# 첫 주문일
first_order_dates = df.groupby('cust_id')['order_date'].min()

# 첫 주문일이 2015년 9월부터 2016년 3월 사이인 고객만 대상으로 분석 진행
start_date = '2015-09-01'
end_date = '2016-03-31'

selected_customers = first_order_dates[(first_order_dates >= start_date) & (first_order_dates <= end_date)].index

df_selected_date = df[df['cust_id'].isin(selected_customers)]
df_selected_date.head()

* 15년 9월~16년 3월 사이에 첫 주문을 한 고객 데이터만 추출 (16년 3월에 첫 구매한 고객이 16년 5월에 재구매시 그 데이터도 포함)
* 1,170,213행
* 857,224명의 고객

In [ ]:
df_selected_date.info()

In [ ]:
# 첫 구매일기준으로 고객 2그룹으로 구분
start_date_1 = pd.to_datetime('2015-09-01')
end_date_1 = pd.to_datetime('2015-11-30')

start_date_2 = pd.to_datetime('2015-12-01')
end_date_2 = pd.to_datetime('2016-03-31')

first_order_dates = df_selected_date.groupby('cust_id')['order_date'].min()


def assign_group(row):
    if start_date_1 <= row < end_date_1:
        return 1
    elif start_date_2 <= row <= end_date_2:
        return 2
    else:
        return None 
    
# 그룹 컬럼 추가
df_selected_date['group'] = df_selected_date['cust_id'].map(first_order_dates).apply(assign_group)
df_selected_date['group'] = df_selected_date['group'].fillna(0).astype(int)
df_selected_date = df_selected_date[df_selected_date['group'] != 0]
df_selected_date.head()

In [ ]:
columns_to_describe = ['gmv', 'units', 'sla', 'product_mrp', 'product_procurement_sla']
# 분포 확인 후 이상치 제거
# group 1
group_1_description = df_selected_date[
    (df_selected_date['group'] == 1) 
    & (df_selected_date['gmv'] <= 5000) 
    & (df_selected_date['units'] <= 3) & (df_selected_date['sla'] <= 15) 
    & (df_selected_date['product_mrp'] <= 6000) & (df_selected_date['product_procurement_sla'] <= 15)
][columns_to_describe].describe()

# group 2
group_2_description = df_selected_date[
    (df_selected_date['group'] == 2) 
    & (df_selected_date['gmv'] <= 5000) 
    & (df_selected_date['units'] <= 3) & (df_selected_date['sla'] <= 15) 
    & (df_selected_date['product_mrp'] <= 6000) & (df_selected_date['product_procurement_sla'] <= 15)
][columns_to_describe].describe()

print("Group 1 Description:\n", group_1_description)
print("\nGroup 2 Description:\n", group_2_description)

* gmv/units/sla/product_mrp/product_procurement_sla에서 그룹 간 차이는 존재하지 않음

In [ ]:
# 그룹 1
group_1_total_products = df_selected_date[df_selected_date['group'] == 1] \
    .groupby('product_analytic_vertical')['units'] \
    .sum()

# 그룹 2
group_2_total_products = df_selected_date[df_selected_date['group'] == 2] \
    .groupby('product_analytic_vertical')['units'] \
    .sum()

# 상위 9개 제품만 
top_9_group_1 = group_1_total_products.nlargest(9)
top_9_group_2 = group_2_total_products.nlargest(9)

# 나머지 제품들을 'Other'로
group_1_other = group_1_total_products.drop(top_9_group_1.index)
group_1_total_products = pd.concat([top_9_group_1, pd.Series({'Other': group_1_other.sum()})])

group_2_other = group_2_total_products.drop(top_9_group_2.index)
group_2_total_products = pd.concat([top_9_group_2, pd.Series({'Other': group_2_other.sum()})])

fig, axes = plt.subplots(1, 2, figsize=(16, 6)) 
# 그룹 1 그래프
axes[0].pie(group_1_total_products, labels=group_1_total_products.index, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.4})
axes[0].set_title('Product Purchase Proportions - Group 1')
# 그룹 2 그래프
axes[1].pie(group_2_total_products, labels=group_2_total_products.index, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.4})
axes[1].set_title('Product Purchase Proportions - Group 2')

plt.tight_layout()
plt.show()

**Group 1에서 많이 구매한 제품**
* LaptopSpeaker / MobileSpeaker / AudioMP3Player 순
* 1위와 2위 제품의 구매비율 차이가 큼 (10%P차이)

**Group 2에서 많이 구매한 제품**
* MobileSpeaker / LaptopSpeaker / PhysicalGame 순
* 2위와 3위 제품의 구매비율 차이가 큼 (약 2배)

In [ ]:
# 할인율 공식을 구하기 위한 units별 평균 gmv/mrp 계산
df_filtered = df_selected_date[df_selected_date['product_analytic_vertical'] == 'LaptopSpeaker']

df_units_summary = df_filtered.groupby('units').agg(
    avg_gmv=('gmv', 'mean')
    ,avg_product_mrp=('product_mrp', 'mean')
).reset_index()

df_units_summary.head()

* gmv = 실제 판매가격 * units
* mrp = 권장소비자가격
* 할인율 = [mrp - (gmv/units)] / mrp

In [ ]:
# 그룹 1
group_1_products = df_selected_date[(df_selected_date['group'] == 1) & 
                                   (df_selected_date['product_analytic_vertical'].isin(['LaptopSpeaker', 'MobileSpeaker', 'AudioMP3Player']))]
# 할인율
group_1_products['discount_rate'] = group_1_products.apply(
    lambda row: (row['product_mrp'] - (row['gmv'] / row['units'])) / row['product_mrp'] * 100
    if row['units'] > 0 and row['product_mrp'] > 0 and row['gmv'] > 0 else 0,
    axis=1
)
# 제품별 평균 할인율
average_discount_rate_per_product1 = group_1_products.groupby('product_analytic_vertical')['discount_rate'].mean()

# 그룹 2
group_2_products = df_selected_date[(df_selected_date['group'] == 2) & 
                                   (df_selected_date['product_analytic_vertical'].isin(['MobileSpeaker','LaptopSpeaker','PhysicalGame']))]
# 할인율 
group_2_products['discount_rate'] = group_2_products.apply(
    lambda row: (row['product_mrp'] - (row['gmv'] / row['units'])) / row['product_mrp'] * 100
    if row['units'] > 0 and row['product_mrp'] > 0 and row['gmv'] > 0 else 0,
    axis=1
)
# 제품별 평균 할인율 
average_discount_rate_per_product2 = group_2_products.groupby('product_analytic_vertical')['discount_rate'].mean()

# 출력
print("Group 1 Average Discount Rates:")
for product, rate in average_discount_rate_per_product1.items():
    print(f"{product}: {rate:.2f}%")
print("\nGroup 2 Average Discount Rates:")
for product, rate in average_discount_rate_per_product2.items():
    print(f"{product}: {rate:.2f}%")

* 그룹1은 많이 구매한 제품 3개의 구매순위와 할인율 순위가 일치하지 않음
* 그룹2는 많이 구매한 제품 3개의 구매순위와 할인율 순위가 일치함
> 그룹1의 가격민감도가 그룹2보다 낮다

In [ ]:
#그룹1
group_1_products = df_selected_date[(df_selected_date['group'] == 1) & 
                                   (df_selected_date['product_analytic_vertical'].isin(['LaptopSpeaker','MobileSpeaker','AudioMP3Player']))]

group_1_products['order_date'] = pd.to_datetime(group_1_products['order_date'])

# 할인율 (product_mrp - (gmv / units)) / product_mrp
group_1_products['discount_rate'] = group_1_products.apply(
    lambda row: (row['product_mrp'] - (row['gmv'] / row['units'])) / row['product_mrp'] * 100
    if row['units'] > 0 and row['product_mrp'] > 0 and row['gmv'] > 0 else 0,
    axis=1
)

group_1_products['month'] = group_1_products['order_date'].dt.to_period('M')  # 'M'은 월별 집계를 의미

# 월별 평균 할인율과 총 구매 수 집계
monthly_stats1 = group_1_products.groupby(['month', 'product_analytic_vertical']).agg(
    avg_discount_rate=('discount_rate', 'mean'),  # 평균 할인율
    total_units_sold=('units', 'sum')  # 총 구매 수 (units 합계)
).reset_index()

monthly_stats1['month'] = monthly_stats1['month'].astype(str)
monthly_stats1 = monthly_stats1.fillna(0)  # NaN 값을 0으로 대체

#그룹2
group_2_products = df_selected_date[(df_selected_date['group'] == 2) & 
                                   (df_selected_date['product_analytic_vertical'].isin(['MobileSpeaker', 'LaptopSpeaker', 'PhysicalGame']))]

group_2_products['order_date'] = pd.to_datetime(group_2_products['order_date'])

# 할인율 (product_mrp - (gmv / units)) / product_mrp
group_2_products['discount_rate'] = group_2_products.apply(
    lambda row: (row['product_mrp'] - (row['gmv'] / row['units'])) / row['product_mrp'] * 100
    if row['units'] > 0 and row['product_mrp'] > 0 and row['gmv'] > 0 else 0,
    axis=1
)

group_2_products['month'] = group_2_products['order_date'].dt.to_period('M')  

# 월별 평균 할인율과 총 구매 수 집계
monthly_stats2 = group_2_products.groupby(['month', 'product_analytic_vertical']).agg(
    avg_discount_rate=('discount_rate', 'mean'),  # 평균 할인율
    total_units_sold=('units', 'sum')  # 총 구매 수 (units 합계)
).reset_index()

monthly_stats2['month'] = monthly_stats2['month'].astype(str)
monthly_stats2 = monthly_stats2.fillna(0)  # NaN 값을 0으로 대체

# 시각화
sns.set(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(18, 12))  

# 그룹1의 월별 평균 할인율
sns.lineplot(data=monthly_stats1, x='month', y='avg_discount_rate', hue='product_analytic_vertical', ax=axes[0, 0], marker='o', palette='Blues')
axes[0, 0].set_title('Group 1 - Monthly Average Discount Rate by Product')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Average Discount Rate (%)')

# 그룹2의 월별 평균 할인율
sns.lineplot(data=monthly_stats2, x='month', y='avg_discount_rate', hue='product_analytic_vertical', ax=axes[0, 1], marker='o', palette='Oranges')
axes[0, 1].set_title('Group 2 - Monthly Average Discount Rate by Product')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Average Discount Rate (%)')

# 그룹1의 월별 총 구매 수
sns.lineplot(data=monthly_stats1, x='month', y='total_units_sold', hue='product_analytic_vertical', ax=axes[1, 0], marker='o', palette='Blues')
axes[1, 0].set_title('Group 1 - Monthly Total Units Sold by Product')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Total Units Sold')

# 그룹2의 월별 총 구매 수
sns.lineplot(data=monthly_stats2, x='month', y='total_units_sold', hue='product_analytic_vertical', ax=axes[1, 1], marker='o', palette='Oranges')
axes[1, 1].set_title('Group 2 - Monthly Total Units Sold by Product')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Total Units Sold')

plt.tight_layout()
plt.show()

**그룹별 구매상위3개 제품 월별 할인율 / 판매수 비교**
* 그룹1 : 할인율과 판매수 추이가 거의 일치하지 않는다 (월별 증감방향이 불일치) / 15년 11월 이후 재구매가 매우 적음
* 그룹2 : 할인율과 판매수 추이가 유사하다 (MobileSpeaker / LaptopSpeaker 월별 증감방향 동일) / 16년 3월 이후 재구매에서는 할인율의 영향이 줄어들었다.

In [ ]:
group_1 = df_selected_date[df_selected_date['group'] == 1]

# 총 매출액
group_1_total_gmv = group_1['gmv'].sum()
# 주문 건수 
group_1_order_count = group_1['order_id'].nunique()
# 고객 수
group_1_cust_count = group_1['cust_id'].nunique()

# 객단가
group_1_avg_order_value = group_1_total_gmv / group_1_order_count
group_1_avg_cust_value = group_1_total_gmv / group_1_cust_count
print(f"Group 1 주문건수 기준 객단가: {group_1_avg_order_value:.2f}, 고객수 기준 객단가: {group_1_avg_cust_value:.2f}")

group_2 = df_selected_date[df_selected_date['group'] == 2]

# 총 매출액 
group_2_total_gmv = group_2['gmv'].sum()
# 주문 건수
group_2_order_count = group_2['order_id'].nunique()
# 고객 수
group_2_cust_count = group_2['cust_id'].nunique()

# 주문건수 기준 객단가
group_2_avg_order_value = group_2_total_gmv / group_2_order_count
group_2_avg_cust_value = group_2_total_gmv / group_2_cust_count
print(f"Group 2 주문건수 기준 객단가: {group_2_avg_order_value:.2f}, 고객수 기준 객단가: {group_2_avg_cust_value:.2f}")

* 그룹1이 주문 1건당 구매금액은 크지만, 고객 1명당 구매금액은 작음 > 주문 1건당 큰 금액을 지불하지만 주문 빈도는 낮음

**Insight**
* 목표 : 그룹1(재구매 적음)을 그룹2처럼 재구매하도록 만들기
* 그룹1은 주문 1건당 평균 주문금액은 크지만 주문 빈도가 낮다
* 그룹1의 가격민감도가 그룹2보다 낮다
________
* 그룹1의 재주문 유도를 위해서는 가격 할인 정책보다 고객충성도정책(등급별 혜택, 보상 프로그램, 맞춤형 제안, 추천 시스템 등)이 효과적일 것이다.
* 가격민감도가 높은 고객은 할인 프로모션 진행시 재구매할 확률이 높다.
 * 그룹1 중에서 가격민감도가 높은 고객 추출 후 해당 고객 대상으로 가격 할인 프로모션 진행하여 재구매 유도


# RFM 분석

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
df_selected_date.head()

In [ ]:
df_selected_date['order_date'] = pd.to_datetime(df_selected_date['order_date'], errors='coerce')
target_date = df_selected_date['order_date'].max()

last_order = df_selected_date.groupby('cust_id')['order_date'].max().reset_index()
last_order.columns = ['cust_id', 'last_order']

df_selected_date = df_selected_date.merge(last_order, on='cust_id')

# Recency (target_date - 마지막 주문일)
df_selected_date['Recency'] = (target_date - df_selected_date['last_order']).dt.days

# Frequency (고객별 주문 수)
frequency = df_selected_date.groupby('cust_id')['order_id'].nunique().reset_index()
frequency.columns = ['cust_id', 'Frequency']

# Monetary (고객별 총 GMV)
monetary = df_selected_date.groupby('cust_id')['gmv'].sum().reset_index()
monetary.columns = ['cust_id', 'Monetary']

RFM = df_selected_date[['cust_id', 'Recency']].drop_duplicates()
RFM = RFM.merge(frequency, on='cust_id')
RFM = RFM.merge(monetary, on='cust_id')

df_selected_date = df_selected_date.merge(RFM[['cust_id', 'Recency', 'Frequency', 'Monetary']], on='cust_id', how='left')

df_selected_date = df_selected_date.drop(columns=['Recency_x']) # 중복컬럼 삭제
df_selected_date = df_selected_date.rename(columns={'Recency_y': 'Recency'})
df_selected_date.head()

In [ ]:
# 이상치 제거
threshold_monetary = RFM['Monetary'].quantile(0.95) 
RFM = RFM[RFM['Monetary'] <= threshold_monetary]

threshold_frequency = RFM['Frequency'].quantile(0.99)
RFM = RFM[RFM['Frequency'] <= threshold_frequency]

RFM.describe()

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(131)
sns.histplot(data=RFM, x = 'Recency', kde=True)
plt.title('Recency')

plt.subplot(132)
sns.histplot(data=RFM, x = 'Frequency', kde=True)
plt.title('Frequency')

plt.subplot(133)
sns.histplot(data=RFM, x = 'Monetary', kde=True)
plt.title('Monetary')

plt.tight_layout()
plt.show()

* R : 약 270일대에 매우 높은 수치 기록 > 15년 10월 신규가입자가 가장 높았던때 유입된 신규고객 중 재구매를 하지 않은 고객이 많음
* F : 대부분 1회 구매에서 최대 4회 구매
* M : 금액이 적을수록 주문 수 많음

In [ ]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(RFM[['Recency', 'Frequency', 'Monetary']])

# 클러스터 개수 결정을 위한 엘보우 포인트 확인
wcss = []
cluster_range = range(1, 11)  # 클러스터 수 1부터 10까지

for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(rfm_scaled)
    wcss.append(kmeans.inertia_)  # WCSS 저장

# WCSS 시각화
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, wcss, marker='o')
plt.title('WCSS by cluster count')
plt.xlabel('cluster count')
plt.ylabel('WCSS')
plt.xticks(cluster_range)
plt.grid()
plt.show()

* 경사가 완만해지는 지점인 4를 클러스터 수로 결정

In [ ]:
# K-means로 군집화
# StandardScaler로 스케일링 후 K-means로 군집화

# K-means 모델 학습
kmeans = KMeans(n_clusters=4, random_state=42) # 엘보우 방법 통해 클러스터 수 4가 적절하다고 판단
RFM['Cluster'] = kmeans.fit_predict(rfm_scaled)

# 각 군집의 RFM 평균값 확인
rfm_cluster_summary = RFM.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'cust_id': 'count'
}).rename(columns={'cust_id': 'Customer_Count'}).reset_index()

print("RFM 군집화 요약:")
rfm_cluster_summary

**Insight**
* 클러스터0 : 최근 구매했지만 단발성 구매로 재구매가 적으며 평균 구매금액이 낮은 고객군
* 클러스터1 : 최근 구매하였으며 구매빈도도 높은 그룹으로 구매 금액도 높은편 (목표 고객군으로 설정)
* 클러스터2 : 과거 신규유입 후 재구매가 없는 고객군으로 구매금액이 적음. 비활성 고객으로 재참여 유도 필요
* 클러스터3 : 과거 신규유입 후 조금 재구매한 고객군으로 고객 수는 적지만 고액을 소비하는 고객군
>클러스터 2와 3을 대상으로 재구매 유도
__________
**가격 민감도 기준**
* 가격 민감도 높은 클러스터: Recency 값이 낮을수록(최근 구매일수록), Frequency 값이 높을수록(주문 빈도 많을수록), Monetary 값이 낮을수록(구매 금액 낮을수록)가격 민감도가 높다.
* 가격 민감도 낮음 클러스터: Recency 값이 높을수록(예전 구매일수록), Frequency 값이 낮을수록(주문 빈도 적을수록), Monetary 값이 높을수록(구매 금액 높을수록)가격 민감도가 낮다.
* 클러스터 0 : 가격 민감도 높음
* 클러스터 1 : 가격 민감도 중간
* 클러스터 2 : 가격 민감도 높음
* 클러스터 3 : 가격 민감도 낮음

**가격 민감도가 높은 클러스터2는 가격 할인 프로모션 / 가격 민감도가 낮은 클러스터3은 고객 충성도 정책 적용**

# 로지스틱 회귀분석

In [ ]:
# 재구매 : cust_id 별로 두 번째 구매일이 존재하면 재구매O / 존재하지 않으면 재구매X
# 재구매여부 컬럼 추가 (재구매 했으면1, 안했으면0)
first_purchase = df_selected_date.groupby('cust_id')['order_date'].min().reset_index()
first_purchase.columns = ['cust_id', 'first_purchase_date']

df_selected_date = df_selected_date.merge(first_purchase, on='cust_id', how='left')

second_purchase = df_selected_date[df_selected_date['order_date'] > df_selected_date['first_purchase_date']]
second_purchase = second_purchase.groupby('cust_id')['order_date'].min().reset_index()
second_purchase.columns = ['cust_id', 'second_purchase_date']

df_selected_date = df_selected_date.merge(second_purchase, on='cust_id', how='left')

df_selected_date['reorder'] = df_selected_date['second_purchase_date'].notnull().astype(int)
df_selected_date[['cust_id', 'reorder']].drop_duplicates().head()

In [ ]:
# 가격 민감도 추가  
# 가격 민감도 = Frequency*할인율
# 할인율 = mrp-(gmv/units) / mrp
df_selected_date['discount_rate'] = (df_selected_date['product_mrp'] - df_selected_date['gmv'] / df_selected_date['units']) / df_selected_date['product_mrp'] * 100

df_selected_date = df_selected_date.rename(columns={'Frequency': 'Frequency_existing'})
df_selected_date = df_selected_date.merge(frequency, on='cust_id', how='left')

df_selected_date['discount_sensitivity'] = df_selected_date['Frequency'] * df_selected_date['discount_rate']

df_selected_date = df_selected_date.drop(columns=['Frequency_existing'])

df_selected_date.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# 특징(X)과 타겟(y) 변수 설정
X = df_selected_date[['discount_sensitivity', 'Recency', 'Monetary', 'Frequency', 'discount_rate']] 
y = df_selected_date['reorder']  # 재구매여부는 이진 타겟 변수 (0 또는 1)

In [ ]:
# Stratified Sampling을 사용하여 데이터 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 데이터 확인
print(f"Training data size: {X_train.shape}")
print(f"Test data size: {X_test.shape}")

In [ ]:
# 결측치 처리 (0으로)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# 무한대 값 처리 (무한대 값 0으로)
X_train.replace([np.inf, -np.inf], 0, inplace=True)
X_test.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
# StandardScaler 사용하여 스케일링
scaler = StandardScaler()

# X_train과 X_test 모두 스케일링
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 로지스틱 회귀 모델 생성
model = LogisticRegression()

# 모델 훈련
model.fit(X_train_scaled, y_train)

# 예측
y_pred = model.predict(X_test_scaled)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))

* 모델 성능이 좋음 / 가격 민감도가 높은 고객이 가격 할인 프로모션 적용시 재구매할 확률이 높다 라는 가설을 지지함

---


## 각 군집별로 "총매출액" 에 중요하게 영향을 미치는 변수를 회귀분석으로 확인하고 각 군집별로 총매출을 늘릴수있는 개선방안을 변수에 대한 해결책을 위주로 인사이트 도출

### 이에 따른 방법으로 타겟변수를 매출과 관련된 변수로 잡고 회귀분석과 feature_importance를 정해 매출과 관련된 최중요 변수를 찾고, 이와의 연관성을 파악한뒤 군집별 매출액 증대 개선안 인사이트를 도출하는 방식을 도입

### 매출액과 직결되는 타겟변수는  monetary변수와 gmv로 나눠볼수있을것같습니다


### 1.타겟변수를 monetary로 잡고하는 방법

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 군집별로 데이터 분리 및 회귀 분석과 feature importance 수행
cluster_results = []

for cluster in df['Cluster'].unique():
    cluster_data = df[df['Cluster'] == cluster]
    
    # 사용할 특성 및 타겟 변수 설정
    X = cluster_data[['Recency', 'Frequency', 'R_Score', 'F_Score', 'M_Score', 'product_mrp', 'deliverybdays']]
    y = cluster_data['Monetary']
    
    # 학습 및 테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # RandomForest를 활용한 feature importance 분석
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    feature_importances = rf.feature_importances_
    
    # 중요 변수 시각화
    feature_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_df)
    plt.title(f'Feature Importance for Cluster {cluster}')
    plt.show()
    
    # Linear Regression 모델 학습
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    
    # 성능 평가
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 회귀 계수 출력
    coefficients = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': lr.coef_
    }).sort_values(by='Coefficient', ascending=False)
    
    print(f"\nCluster {cluster} - Linear Regression Coefficients:")
    print(coefficients)
    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2}")
    
    # 결과 저장
    cluster_results.append({
        'Cluster': cluster,
        'Feature_Importance': feature_df,
        'Regression_Coefficients': coefficients,
        'MSE': mse,
        'R2': r2
    })

# 각 군집별로 feature importance와 회귀 분석 결과를 분석합니다.


### 2.타겟변수를  gmv로 계산하는 방법

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 군집별로 데이터 분리 및 회귀 분석과 feature importance 수행
cluster_results = []

for cluster in df['Cluster'].unique():
    cluster_data = df[df['Cluster'] == cluster]
    
    # 사용할 특성 및 타겟 변수 설정
    X = cluster_data[['Recency', 'Frequency', 'Monetary', 'product_mrp', 'discount_rate', 'units']]
    y = cluster_data['gmv']
    
    # 학습 및 테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # RandomForest를 활용한 feature importance 분석
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    feature_importances = rf.feature_importances_
    
    # 중요 변수 시각화
    feature_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_df)
    plt.title(f'Feature Importance for Cluster {cluster}')
    plt.show()
    
    # Linear Regression 모델 학습
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    
    # 성능 평가
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 회귀 계수 출력
    coefficients = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': lr.coef_
    }).sort_values(by='Coefficient', ascending=False)
    
    print(f"\nCluster {cluster} - Linear Regression Coefficients:")
    print(coefficients)
    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2}")
    
    # 결과 저장
    cluster_results.append({
        'Cluster': cluster,
        'Feature_Importance': feature_df,
        'Regression_Coefficients': coefficients,
        'MSE': mse,
        'R2': r2
    })

# 각 군집별로 feature importance와 회귀 분석 결과를 분석합니다.


- 1. 군집 0 (Cluster 0)
주요 변수: Frequency, product_mrp, Monetary
Frequency와 product_mrp의 중요도가 높게 나타났으며, 회귀 계수에서도 양의 상관관계를 보였습니다. 이는 고객이 자주 구매할수록 총매출이 높아진다는 점을 시사합니다.
product_mrp는 제품의 권장소비자가격으로, 높은 가격대의 제품을 구매할수록 총매출이 증가하는 결과를 보였습니다.
개선 방안:
구매 빈도를 늘릴 수 있도록 구독형 프로그램이나 포인트 적립 프로그램을 도입하여 고객의 재구매 유도를 강화할 수 있습니다.
고급 제품에 대한 홍보와 할인 프로모션을 통해 고객이 고가의 제품에 더 많은 관심을 가지도록 유도할 수 있습니다.

- 2. 군집 1 (Cluster 1)
주요 변수: units, discount_rate, Recency
units가 높은 중요도를 보였으며, 이는 구매 수량이 많을수록 총매출이 증가한다는 점을 반영합니다.
discount_rate는 회귀 계수에서 양의 관계로 나타나, 할인율이 높을 때 총매출이 증가하는 경향을 보였습니다. 이는 이 군집이 할인에 민감하다는 것을 시사합니다.
개선 방안:
할인 프로모션을 적극적으로 활용하여 다량 구매를 유도합니다. 예를 들어, 1+1 할인이나 대량 구매 시 추가 할인 등의 프로모션이 효과적일 수 있습니다.
재구매를 활성화하기 위해 구매 후 일정 기간 동안 할인 쿠폰을 제공하는 것도 효과적입니다.

3. 군집 2 (Cluster 2)
주요 변수: discount_rate, Frequency, Recency
discount_rate가 높은 상관관계를 보였으며, 할인율이 증가할 때 총매출도 함께 증가하는 경향을 나타냈습니다. 이는 할인에 매우 민감한 고객군이라는 점을 시사합니다.
또한, Recency가 높은 중요도를 보여 고객의 최신 구매 이력이 총매출에 영향을 미친다는 점이 확인되었습니다.
개선 방안:
이 군집은 할인 프로모션이 중요한 역할을 하므로 정기적인 할인 행사를 통해 구매를 촉진할 수 있습니다.
리타겟팅 캠페인을 통해 일정 기간 내 구매 이력이 있는 고객에게 추가 할인을 제공하여 재구매를 유도할 수 있습니다.

4. 군집 3 (Cluster 3)
주요 변수: Monetary, product_mrp, Recency
Monetary는 군집 3에서 매우 중요한 변수로 나타났으며, 총매출에 긍정적인 영향을 미쳤습니다.
product_mrp도 중요한 변수로, 고가 제품의 구매가 매출에 크게 기여하고 있음을 보여줍니다.
개선 방안:
이 군집은 고급 제품 선호도가 높으므로 고급 제품 추천이나 프리미엄 고객 대상의 전용 혜택을 제공하여 고가 제품 구매를 더욱 활성화할 수 있습니다.
프리미엄 구독 서비스를 제공하여 장기적으로 이탈을 줄이고 재구매율을 높일 수 있는 방안을 고려할 수 있습니다.





요약 및 인사이트
각 군집에서 총매출에 중요한 변수들이 다르게 나타났으며, 군집별 맞춤형 전략이 효과적일 것으로 판단됩니다.

할인 민감도가 높은 군집(Cluster 1, 2)에서는 정기적 할인과 대량 구매 혜택을 제공하여 구매를 촉진할 수 있습니다.
고가 제품을 선호하는 군집(Cluster 0, 3)에서는 고급 제품 홍보 및 프리미엄 서비스로 매출을 증대할 수 있는 기회를 찾을 수 있습니다.

# 가격 할인 정책 적용할 제품 분석

In [ ]:
# 클러스터 2 고객들만 추출
df_cluster2 = df_selected_date[df_selected_date['group'] == 2]
df_cluster2.info()

In [ ]:
# 결측치 처리
df_cluster2_clean = df_cluster2.dropna(subset=['discount_rate'])
df_cluster2_clean = df_cluster2_clean[~df_cluster2_clean['discount_rate'].isin([0, -np.inf, np.inf])]

df_cluster2_clean.info()

In [ ]:
df_cluster2_clean['order_date'] = pd.to_datetime(df_cluster2_clean['order_date'])
df_cluster2_clean['year_month'] = df_cluster2_clean['order_date'].dt.to_period('M')
df_cluster2_clean[['order_date', 'year_month']].head()

In [ ]:
# 제품 카테고리별 월별 평균 할인율 및 주문 수 계산
monthly_data = df_cluster2_clean.groupby(['year_month', 'product_analytic_category']).agg(
    average_discount=('discount_rate', 'mean'),  # 평균 할인율
    order_count=('order_id', 'nunique')          # 월별 주문 수 
).reset_index()
monthly_data.head()

In [ ]:
if pd.api.types.is_period_dtype(monthly_data['year_month']):
    monthly_data['year_month'] = monthly_data['year_month'].dt.to_timestamp()

monthly_data['average_discount'] = pd.to_numeric(monthly_data['average_discount'], errors='coerce')

monthly_data['order_count'] = pd.to_numeric(monthly_data['order_count'], errors='coerce')

# 제품 카테고리별 월별 평균 할인율 추이
plt.figure(figsize=(14, 8))
sns.lineplot(x='year_month', y='average_discount', hue='product_analytic_category', data=monthly_data, marker='o')
plt.title('Monthly Average Discount Rate by Product Category (Cluster 2)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Average Discount Rate', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Product Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()  # 레이아웃 조정
plt.show()

# 제품 카테고리별 월별 주문 수 추이
plt.figure(figsize=(14, 8))
sns.lineplot(x='year_month', y='order_count', hue='product_analytic_category', data=monthly_data, marker='o')
plt.title('Monthly Order Count by Product Category (Cluster 2)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Order Count', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Product Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()  # 레이아웃 조정
plt.show()

* CameraAccessory가 할인율과 주문수 추이가 가장 유사함

In [ ]:
# CameraAccessory 카테고리
camera_accessory_data = df_cluster2_clean[df_cluster2_clean['product_analytic_category'] == 'CameraAccessory']

monthly_subcategory_data = camera_accessory_data.groupby(['year_month', 'product_analytic_sub_category']).agg(
    average_discount=('discount_rate', 'mean'),  
    order_count=('order_id', 'nunique')  
).reset_index()

if monthly_subcategory_data['year_month'].dtype == 'period[M]':  # 'M'은 월 단위 Period
    monthly_subcategory_data['year_month'] = monthly_subcategory_data['year_month'].dt.to_timestamp()

# 서브 카테고리별 월별 평균 할인율 추이
plt.figure(figsize=(14, 8))
sns.lineplot(x='year_month', y='average_discount', hue='product_analytic_sub_category', data=monthly_subcategory_data, marker='o')
plt.title('Monthly Average Discount Rate by Product Subcategory (CameraAccessory)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Average Discount Rate', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Subcategory', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 서브 카테고리별 월별 총 주문 수 추이
plt.figure(figsize=(14, 8))
sns.lineplot(x='year_month', y='order_count', hue='product_analytic_sub_category', data=monthly_subcategory_data, marker='o')
plt.title('Monthly Order Count by Product Subcategory (CameraAccessory)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Order Count', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Subcategory', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

* 가격 민감도가 높은 클러스터2 고객군을 대상으로 CameraAccessory 제품 할인 프로모션 진행시 재구매율 증가에 도움이 될 것이다.